In [3]:
import webdataset as wds
import jax
import jax.numpy as jnp
import augmax
import matplotlib.pyplot as plt

import grain.python as pygrain
from typing import Any, Dict, List, Tuple
import numpy as np
from functools import partial
import tqdm 

import fsspec
import json

import os
from transformers import AutoTokenizer, FlaxCLIPTextModel, CLIPTextModel

from datasets import load_dataset, concatenate_datasets
from datasets.utils.file_utils import get_datasets_user_agent
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib

import PIL.Image
import cv2

/home/mrwhite0racle/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
USER_AGENT = get_datasets_user_agent()


def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image

denormalizeImage = lambda x: (x + 1.0) * 127.5

def plotImages(imgs, fig_size=(8, 8), dpi=100):
    fig = plt.figure(figsize=fig_size, dpi=dpi)
    imglen = imgs.shape[0]
    for i in range(imglen):
        plt.subplot(fig_size[0], fig_size[1], i + 1)
        plt.imshow(jnp.astype(denormalizeImage(imgs[i, :, :, :]), jnp.uint8))
        plt.axis("off")
    plt.show()


# Filtering pipeline for various datasets

In [5]:
def dataMapper(map: Dict[str, Any]):
    def _map(sample) -> Dict[str, Any]:
        return {
            "url": sample[map["url"]],
            "caption": sample[map["caption"]],
        }
    return _map

def imageFetcher():
    def fetch_images(batch, num_threads, timeout=None, retries=0):
        fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            batch["image"] = list(executor.map(fetch_single_image_with_args, batch["url"]))
        return batch
    return fetch_images

def mapDataset(dataset, args, mapper=dataMapper, workers=16, batch_size=10000, should_remove_columns=True, fn_kwargs={}):
    if should_remove_columns:
        remove_columns = dataset.column_names
    else:
        remove_columns = None
    return dataset.map(mapper(*args), batched=True, batch_size=batch_size, remove_columns=remove_columns, num_proc=workers, fn_kwargs=fn_kwargs) 

In [5]:
laion12m6 = load_dataset("dclure/laion-aesthetics-12m-umap")
laion12m6_fused = laion12m6['train']
laionMap = {
    "url": "URL",
    "caption": "TEXT",
}
laion12m6_fused = mapDataset(laion12m6_fused, (laionMap, ))

Map (num_proc=16): 100%|██████████| 12096809/12096809 [00:49<00:00, 246633.91 examples/s] 


In [6]:
mscoco = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")
mscoco_fused = mscoco['train']
mscocoMap = laionMap
mscoco_fused = mapDataset(mscoco_fused, (mscocoMap, ))

Map (num_proc=16): 100%|██████████| 591753/591753 [00:02<00:00, 262603.89 examples/s] 


In [7]:
fused_data = concatenate_datasets([laion12m6_fused, mscoco_fused, mscoco_fused])

In [8]:
len(fused_data)

12688562

In [ ]:
for i in range(0, 20):
    sample = fused_data[i]
    img = fetch_single_image(sample['url'])
    if img is None:
        print("Image is None")
        continue
    text = sample['caption']
    plt.imshow(img)
    plt.title(text)
    # print(f"Aesthetic score: {sample['aesthetic_score_laion_v2']}")
    plt.show()

In [9]:
fused_data.to_parquet("laion-aesthetics-12m+mscoco-2017.parquet")

Creating parquet from Arrow format: 100%|██████████| 12689/12689 [00:20<00:00, 622.45ba/s] 


2454709042

In [118]:
imaged_data = mapDataset(mscoco_fused, (), mapper=imageFetcher, batch_size=5000, workers=64, should_remove_columns=False, fn_kwargs={"num_threads": 64})

/home/mrwhite0racle/.local/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=64):  19%|█▊        | 110000/591753 [01:47<03:12, 2506.78 examples/s]

# COYO-700M Processing

In [6]:
coyo700 = load_dataset("kakaobrain/coyo-700m", num_proc=32)

In [7]:
baseFilterMap = {
    # "word_count": {"min": 0, "max": 100},
    "clip_similarity_vitl14": {"min": 0.27, "max": 1000},
    "aesthetic_score_laion_v2": {"min": 5.1, "max": 100},
    "watermark_score": {"min": 0, "max": 0.4},
}

heavyFilterMap = {
    # "word_count": {"min": 0, "max": 100},
    "clip_similarity_vitl14": {"min": 0.26, "max": 100},
    "aesthetic_score_laion_v2": {"min": 5.0, "max": 100},
    "watermark_score": {"min": 0, "max": 0.8},
    "width": {"min":512, "max":99999},
    "height": {"min":512, "max":99999},
}

def coyoFilter(filterMap):
    def _filter(sample):
        for key, value in filterMap.items():
            if sample[key] < value["min"] or sample[key] > value["max"]:
                return False
        return True
    return _filter
    

In [8]:
# goodCoyo700 = coyo700.filter(coyoFilter(baseFilterMap), num_proc=64)
aestheticCoyo700 = coyo700.filter(coyoFilter(heavyFilterMap), num_proc=64)

Filter (num_proc=64): 100%|██████████| 746972269/746972269 [03:33<00:00, 3505937.43 examples/s]


In [17]:
len(aestheticCoyo700['train'])

16586129

In [ ]:
for i in range(0, 100):
    sample = coyo700Filtered['train'][i]
    img = fetch_single_image(sample['url'])
    if img is None:
        print("Image is None")
        continue
    text = sample['text']
    plt.imshow(img)
    plt.title(text)
    print(f"Aesthetic score: {sample['aesthetic_score_laion_v2']}")
    plt.show()

In [19]:
final_data = mapDataset(aestheticCoyo700['train'], ({
    "url":"url",
    "caption":"text"
    }, ))

Map (num_proc=16): 100%|██████████| 16586129/16586129 [01:15<00:00, 218835.67 examples/s]


In [20]:
final_data.to_parquet("aestheticCoyo_0.26_clip_5.5aesthetic_256plus.parquet")

Creating parquet from Arrow format: 100%|██████████| 16587/16587 [00:22<00:00, 729.06ba/s]


3343770169

In [17]:
final_data[0]

{'url': 'https://img3.goodfon.com/wallpaper/big/5/85/art-krenz-fallen-angel-angel.jpg',
 'caption': 'Picture girl, fiction, fire, wings, angel, red eyes, white hair, art, Angel Fall, Krenz'}